# ポートフォリオ最適化

In [1]:
from qiskit import Aer
from qiskit.circuit.library import TwoLocal
from qiskit.aqua import QuantumInstance
from qiskit.finance.applications.ising import portfolio
from qiskit.optimization.applications.ising.common import sample_most_likely
from qiskit.finance.data_providers import RandomDataProvider
from qiskit.aqua.algorithms import VQE, QAOA, NumPyMinimumEigensolver
from qiskit.aqua.components.optimizers import COBYLA
import numpy as np
import matplotlib.pyplot as plt
import datetime
import pandas as pd
import pandas_datareader.data as web

In [2]:
# set number of assets (= number of qubits)
num_assets = 4

## ランダムな時系列データを用意

In [3]:
# # Generate expected return and covariance matrix from (random) time-series
# stocks = [("TICKER%s" % i) for i in range(num_assets)]
# data = RandomDataProvider(tickers=stocks,
#                  start=datetime.datetime(2016,1,1),
#                  end=datetime.datetime(2016,1,30))
# data.run()
# mu = data.get_period_return_mean_vector()
# sigma = data.get_period_return_covariance_matrix()

## 実際の時系列データを用いる

In [4]:
!pip install pandas pandas_datareader

In [5]:
# 銘柄選択
codes = ["GOOG", 'AAPL', 'FB', 'AMZN'] # GAFA

# 2017年の1年間のデータを使用
start = datetime.datetime(2017, 1, 1)
end = datetime.datetime(2017, 12, 31)

# Yahoo! Financeから日次の株価データを取得
data = web.DataReader(codes, 'yahoo', start, end)

df = data['Adj Close'] 

## 直近のデータの表示
display(df.tail())

Symbols,GOOG,AAPL,FB,AMZN
Date,,,,
2017-12-22,1060.119995,42.040852,177.199997,1168.359985
2017-12-26,1056.739990,40.974274,175.990005,1176.760010
2017-12-27,1049.369995,40.981483,177.619995,1182.260010
2017-12-28,1048.140015,41.096786,177.919998,1186.099976
2017-12-29,1046.400024,40.652374,176.460007,1169.469971


In [6]:
daily_return = df.pct_change() # 日次リターン
expected_return = daily_return.dropna(how='all').mean() * 252 # 期待リターン：年率換算のため年間の営業日数252を掛ける
cov = daily_return.dropna(how='all').cov() * 252 # 年率換算のため

In [7]:
q = 1                   # set risk factor
budget = num_assets  # set budget (the number of assets to be selected out of n)
penalty = num_assets      # set parameter to scale the budget penalty term

qubitOp, offset = portfolio.get_operator(expected_return.values, cov.values, q, budget, penalty)

In [8]:
def index_to_selection(i, num_assets):
    s = "{0:b}".format(i).rjust(num_assets)
    x = np.array([1 if s[i]=='1' else 0 for i in reversed(range(num_assets))])
    return x

def print_result(result):
    selection = sample_most_likely(result.eigenstate)
    value = portfolio.portfolio_value(selection, expected_return.values, cov.values, q, budget, penalty)
    np.set_printoptions(formatter={'float': '{:.4f}'.format})
    print('Optimal: selection {}, value {}'.format(selection, value))

    eigenvector = result.eigenstate if isinstance(result.eigenstate, np.ndarray) else result.eigenstate.to_matrix()
    probabilities = np.abs(eigenvector)**2
    i_sorted = reversed(np.argsort(probabilities))
    print('\n----------------- Full result ---------------------')
    print('selection\tvalue\t\tprobability')
    print('---------------------------------------------------')
    for i in i_sorted:
        x = index_to_selection(i, num_assets)
        value = portfolio.portfolio_value(x, expected_return.values, cov.values, q, budget, penalty)
        probability = probabilities[i]
        print('%10s\t%.4f\t\t%.4f' %(x, value, probability))

In [9]:
backend = Aer.get_backend('statevector_simulator')
seed = 50

cobyla = COBYLA()
cobyla.set_options(maxiter=500)
ry = TwoLocal(qubitOp.num_qubits, 'ry', 'cz', reps=3, entanglement='full')
vqe = VQE(qubitOp, ry, cobyla)
vqe.random_seed = seed

quantum_instance = QuantumInstance(backend=backend, seed_simulator=seed, seed_transpiler=seed)

result = vqe.run(quantum_instance)

print_result(result)

Optimal: selection [1.0000 1.0000 1.0000 1.0000], value -1.2547346932627472

----------------- Full result ---------------------
selection	value		probability
---------------------------------------------------
 [1 1 1 1]	-1.2547		0.9999
 [1 1 0 1]	3.0305		0.0000
 [1 0 1 1]	3.0284		0.0000
 [0 1 1 0]	15.2512		0.0000
 [1 0 1 0]	15.3589		0.0000
 [0 1 1 1]	2.9152		0.0000
 [0 0 1 1]	15.2249		0.0000
 [0 0 0 1]	35.5796		0.0000
 [1 0 0 1]	15.3463		0.0000
 [1 1 0 0]	15.3700		0.0000
 [0 1 0 1]	15.2372		0.0000
 [0 0 0 0]	64.0000		0.0000
 [1 1 1 0]	3.0381		0.0000
 [0 0 1 0]	35.5987		0.0000
 [0 1 0 0]	35.6199		0.0000
 [1 0 0 0]	35.7235		0.0000
